In [10]:
import pandas as pd
df = pd.read_csv("10SecondStepPriceEvolution2.csv")
df["F10"] = df["F10"] - df["Cur"]
df["P10"] = df["P10"] - df["Cur"]
df["P20"] = df["P20"] - df["Cur"]
df["P30"] = df["P30"] - df["Cur"]
df["P40"] = df["P40"] - df["Cur"]
df["P50"] = df["P50"] - df["Cur"]
df["P60"] = df["P60"] - df["Cur"]
df.drop('Cur', axis=1, inplace=True)
df['F10'] = df['F10'].apply(lambda x: 1 if x >= 1 else -1 if x <= -1 else 0)

In [11]:
zeros = df[df['F10'] == 0]
ones  = df[df['F10'] == 1]
negas = df[df['F10'] == -1]
zeros = zeros.sample(frac=1).reset_index(drop=True)
zeros = zeros[:15000]
df = pd.concat([zeros, ones, negas])
df = df.sample(frac=1).reset_index(drop=True)

In [12]:
train_dataset = df.head(40000)
test_dataset = df.tail(df.shape[0] - 40000)

In [13]:
import os
import pandas as pd
from torch.utils.data import Dataset

class CustomPriceDataset(Dataset):
    def __init__(self, dataframe, labels, transform=None, target_transform=None):
        self.dataframe = dataframe
        self.labels = labels
        self.transform = transform
        self.target_transform = target_transform
        

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        data = list(self.dataframe.iloc[idx])
        label = self.labels.iloc[idx]
        if self.transform:
            data = self.transform(data)
        if self.target_transform:
            label = self.target_transform(label)
        return data, label

In [14]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Lambda

trnlbl = train_dataset['F10'] + 1
trnset = train_dataset.drop('F10', axis=1)
trainPrice = CustomPriceDataset(trnset, trnlbl, torch.FloatTensor, Lambda(lambda y: torch.zeros(3, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)))
train_dataloader = DataLoader(trainPrice, batch_size=64)

tstlbl = test_dataset['F10'] + 1
tstset = test_dataset.drop('F10', axis=1)
testPrice = CustomPriceDataset(tstset, tstlbl, torch.FloatTensor)
test_dataloader = DataLoader(testPrice, batch_size=64)

In [15]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(6, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 3),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [16]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [17]:
learning_rate = 1e-2
passed = 0

In [24]:
epochs = 100
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=1e-2, dampening=1e-3)

In [25]:
print("Start")
for t in range(epochs):
    print(f"Epoch {t+1 + passed}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
passed = passed + epochs
print("Done!")

Start
Epoch 301
-------------------------------
loss: 0.600904  [   64/40000]
loss: 0.781481  [ 6464/40000]
loss: 0.663539  [12864/40000]
loss: 0.680174  [19264/40000]
loss: 0.771671  [25664/40000]
loss: 0.797513  [32064/40000]
loss: 0.763069  [38464/40000]
Test Error: 
 Accuracy: 46.1%, Avg loss: 1.258042 

Epoch 302
-------------------------------
loss: 0.598505  [   64/40000]
loss: 0.768666  [ 6464/40000]
loss: 0.683601  [12864/40000]
loss: 0.755257  [19264/40000]
loss: 0.819121  [25664/40000]
loss: 0.808160  [32064/40000]
loss: 0.748667  [38464/40000]
Test Error: 
 Accuracy: 46.7%, Avg loss: 1.245972 

Epoch 303
-------------------------------
loss: 0.594879  [   64/40000]
loss: 0.742540  [ 6464/40000]
loss: 0.665287  [12864/40000]
loss: 0.722841  [19264/40000]
loss: 0.773136  [25664/40000]
loss: 0.805300  [32064/40000]
loss: 0.794349  [38464/40000]
Test Error: 
 Accuracy: 47.0%, Avg loss: 1.220536 

Epoch 304
-------------------------------
loss: 0.626308  [   64/40000]
loss: 0.83

loss: 0.760102  [ 6464/40000]
loss: 0.608523  [12864/40000]
loss: 0.631079  [19264/40000]
loss: 0.749039  [25664/40000]
loss: 0.795561  [32064/40000]
loss: 0.746589  [38464/40000]
Test Error: 
 Accuracy: 47.6%, Avg loss: 1.286434 

Epoch 329
-------------------------------
loss: 0.604409  [   64/40000]
loss: 0.781520  [ 6464/40000]
loss: 0.633487  [12864/40000]
loss: 0.627800  [19264/40000]
loss: 0.751348  [25664/40000]
loss: 0.733540  [32064/40000]
loss: 0.735726  [38464/40000]
Test Error: 
 Accuracy: 45.2%, Avg loss: 1.410652 

Epoch 330
-------------------------------
loss: 0.560131  [   64/40000]
loss: 0.704610  [ 6464/40000]
loss: 0.627247  [12864/40000]
loss: 0.751291  [19264/40000]
loss: 0.775087  [25664/40000]
loss: 0.699841  [32064/40000]
loss: 0.753096  [38464/40000]
Test Error: 
 Accuracy: 45.7%, Avg loss: 1.345993 

Epoch 331
-------------------------------
loss: 0.531974  [   64/40000]
loss: 0.713446  [ 6464/40000]
loss: 0.600886  [12864/40000]
loss: 0.698213  [19264/40000

loss: 0.638085  [ 6464/40000]
loss: 0.550637  [12864/40000]
loss: 0.580959  [19264/40000]
loss: 0.766066  [25664/40000]
loss: 0.672767  [32064/40000]
loss: 0.673276  [38464/40000]
Test Error: 
 Accuracy: 45.8%, Avg loss: 1.516442 

Epoch 356
-------------------------------
loss: 0.519005  [   64/40000]
loss: 0.708425  [ 6464/40000]
loss: 0.597083  [12864/40000]
loss: 0.684393  [19264/40000]
loss: 0.774106  [25664/40000]
loss: 0.669984  [32064/40000]
loss: 0.714333  [38464/40000]
Test Error: 
 Accuracy: 46.4%, Avg loss: 1.425636 

Epoch 357
-------------------------------
loss: 0.490168  [   64/40000]
loss: 0.689890  [ 6464/40000]
loss: 0.555746  [12864/40000]
loss: 0.635341  [19264/40000]
loss: 0.708933  [25664/40000]
loss: 0.674699  [32064/40000]
loss: 0.716176  [38464/40000]
Test Error: 
 Accuracy: 46.8%, Avg loss: 1.405545 

Epoch 358
-------------------------------
loss: 0.550493  [   64/40000]
loss: 0.671842  [ 6464/40000]
loss: 0.549139  [12864/40000]
loss: 0.537580  [19264/40000

loss: 0.666759  [ 6464/40000]
loss: 0.497674  [12864/40000]
loss: 0.648621  [19264/40000]
loss: 0.690217  [25664/40000]
loss: 0.693811  [32064/40000]
loss: 0.661010  [38464/40000]
Test Error: 
 Accuracy: 46.0%, Avg loss: 1.606865 

Epoch 383
-------------------------------
loss: 0.501853  [   64/40000]
loss: 0.729638  [ 6464/40000]
loss: 0.479158  [12864/40000]
loss: 0.556068  [19264/40000]
loss: 0.666011  [25664/40000]
loss: 0.699025  [32064/40000]
loss: 0.698043  [38464/40000]
Test Error: 
 Accuracy: 46.3%, Avg loss: 1.582627 

Epoch 384
-------------------------------
loss: 0.497909  [   64/40000]
loss: 0.699561  [ 6464/40000]
loss: 0.490787  [12864/40000]
loss: 0.622798  [19264/40000]
loss: 0.683124  [25664/40000]
loss: 0.643714  [32064/40000]
loss: 0.621725  [38464/40000]
Test Error: 
 Accuracy: 45.8%, Avg loss: 1.624359 

Epoch 385
-------------------------------
loss: 0.511155  [   64/40000]
loss: 2.109704  [ 6464/40000]
loss: 0.508904  [12864/40000]
loss: 0.610685  [19264/40000